In [6]:
! pip install -qU yt_dlp pydub python-dotenv langchain langchain_openai gradio langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.9 MB/s eta 0:00:00


In [11]:
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

True

In [12]:
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import OpenAIWhisperParser
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader

In [13]:
url = "https://www.youtube.com/watch?v=ZiwogMtbjr4&ab_channel=StanfordOnline"
save_dir="docs/youtube/"
loader = GenericLoader(
    YoutubeAudioLoader([url],save_dir),
    OpenAIWhisperParser()
)

In [14]:
docs = loader.load()

[youtube] Extracting URL: https://www.youtube.com/watch?v=ZiwogMtbjr4&ab_channel=StanfordOnline
[youtube] ZiwogMtbjr4: Downloading webpage
[youtube] ZiwogMtbjr4: Downloading ios player API JSON
[youtube] ZiwogMtbjr4: Downloading mweb player API JSON
[youtube] ZiwogMtbjr4: Downloading m3u8 information
[info] ZiwogMtbjr4: Downloading 1 format(s): 140
[download] docs/youtube//General Intro ｜ Stanford CS221： Artificial Intelligence： Principles and Techniques (Autumn 2021).m4a has already been downloaded
[download] 100% of   63.11MiB
[ExtractAudio] Not converting audio docs/youtube//General Intro ｜ Stanford CS221： Artificial Intelligence： Principles and Techniques (Autumn 2021).m4a; file is already in target format m4a
Transcribing part 1!
Transcribing part 2!
Transcribing part 3!
Transcribing part 4!


In [17]:
docs[0].metadata

{'source': 'docs/youtube/General Intro ｜ Stanford CS221： Artificial Intelligence： Principles and Techniques (Autumn 2021).m4a',
 'chunk': 0}

In [18]:
len(docs)

4

In [19]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

<ipython-input-19-ca2f04320657>:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()


In [32]:
sentence_1="I like mango"
sentence_2="I love mango"
sentence_3="Tesla is very costly"

In [33]:
embedding_1 = embedding.embed_query(sentence_1)
embedding_2 = embedding.embed_query(sentence_2)
embedding_3 = embedding.embed_query(sentence_3)

In [34]:
import numpy as np

In [35]:
np.dot(embedding_1,embedding_2)

0.9809540264078283

In [36]:
np.dot(embedding_1,embedding_3)

0.7101333782121164

In [37]:
np.dot(embedding_2,embedding_3)

0.7071724742750878

In [39]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150
)

chunks = text_splitter.split_documents(docs)
len(chunks)

52

In [41]:
! pip install -qU chromadb

In [42]:
persist_directory = "docs/chroma"

In [43]:
from langchain.vectorstores import Chroma

vectordb = Chroma.from_documents(
    documents=chunks,
    embedding = OpenAIEmbeddings(),
    persist_directory=persist_directory
)

In [44]:
vectordb._collection.count()

52

In [45]:
question = "What is the project that is mentioned in the above vedio"

In [46]:
docs = vectordb.similarity_search(question,k=3)

In [47]:
docs

[Document(metadata={'chunk': 0, 'source': 'docs/youtube/General Intro ｜ Stanford CS221： Artificial Intelligence： Principles and Techniques (Autumn 2021).m4a'}, page_content="will be assigned one. Okay. So let me talk about the project also real quick. So the project this quarter is going to be optional. This is what we did last year too, because the course is virtual and we thought it might be a little bit more difficult to find a team and work together. But regardless, a lot of students did the project last year, and there are a lot of interesting ideas and projects came out of that. And it was really exciting to see like so many cool projects like during that quarter too. So I do recommend that you guys look into this closely even though it is optional. So the idea is you want to choose a task where you can actually apply some of the ideas that you have learned as part of this class and use those techniques for that particular task. It's a little bit open-ended. You need to like deci

In [48]:
vectordb.persist()

<ipython-input-48-d832e715b29a>:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [50]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

In [53]:
retrieval_qa = RetrievalQA.from_chain_type(
    llm = ChatOpenAI(model="gpt-4o-mini",temperature=0.0),
    chain_type="stuff",
    retriever = vectordb.as_retriever()
)

In [55]:
retrieval_qa.invoke("What is the project that is mentioned here")

{'query': 'What is the project that is mentioned here',
 'result': 'The project mentioned is optional and allows students to choose a task where they can apply the AI techniques learned in the class. Students can work in groups of up to four people and must complete a set of milestones, including a project interest form, proposal, progress report, video, and final report. The project is open-ended, meaning students can pick any task to work on, and they will have a CA assigned to their group to assist them throughout the process.'}